<a href="https://colab.research.google.com/github/gukouk176-debug/colab2/blob/main/DataScience_06_ipynb_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第6回講義 データの前処理
+ irisを使ったデータ処理
+ タイタニックデータを用いたデータ処理
+ breast-cancerのデータセットのデータ処理

# 全講義共通初期設定
+ 警告の非表示(実装時は非推奨)
+ numpy pandas小数点以下桁数の表示設定
+ pandas全データ表示設定
+ Google driveへの接続

In [ ]:
# ワーニングを非表示にする
# この設定は不都合が見えなくなる為、お勧めしない
# 今回は教育資料用に、出力を簡素化する為に利用する
import warnings
warnings.simplefilter('ignore')

# モジュールの読み込み
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# 小数点以下桁数の表示設定
np.set_printoptions(precision = 3)
pd.options.display.precision = 3

# pandasの全データ表示設定
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
#googledriveに接続
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#google driveと接続できたかを確認
!ls drive/MyDrive/DataScience

# irisを使ったデータ処理

Iris(アヤメのデータ)のデータセットを用いたデータ処理  
ここでは以下の項目について演習を行う。  
+ データの読み込み
+ データの確認
+ 欠損値の確認
+ 説明変数・目的変数の設定
+ データの分割（トレーニングデータ・テストデータ）
+ モデルへの適用
+ 結果の算出
+ ダミー変数化

## データの読み込み

In [ ]:
#ライブラリのインポートとirisデータの読み込み
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

##ライブラリpandasを使ったcsvデータの読み込み,sepで区切り記号を設定,もし日本語の場合,encodingを設定する必要があり
iris_df = pd.read_csv('/content/drive/MyDrive/DataScience/iris.csv',sep=',')

## データの確認

In [ ]:
# 先頭の10行だけ表示したい場合
print(iris_df.head(10))

In [ ]:
# 最後の5行だけ表示したい場合
print(iris_df.tail())

#データセットの項目のデータ型を確認
iris_df.dtypes

In [ ]:
#データセットの項目(列ラベルともいう)確認
iris_df.columns

## 欠損値の確認

In [ ]:
#データセットの情報の表示（各項目の欠損でないデータ数、型など）
print(iris_df.info())
print()

#各項目の欠損値数の表示
print(iris_df.isnull().sum())

In [ ]:
#欠損値の確認、Falseなら欠損値ではない。Trueなら欠損値
print(iris_df.head().isnull())

## 説明変数・目的変数の設定

In [ ]:
#説明変数の設定
#pandasのデータフレームのまま説明変数切り取り
X = iris_df[['sepal.length', 'sepal.width', 'petal.length', 'petal.width']]
#取り出し方 loc:ラベル（変数名）指定 ,iloc:番号指定
#X = iris_df.iloc[:, [0,1,2,3]]
#X = iris_df.loc[:, ['sepal.length','sepal.width','petal.length','petal.width']]

#目的変数の設定
y = iris_df[['species']]
# y = iris_df.iloc[:,[4]]
# y = iris_df.loc[:, ['species']]

#データフレームをarrayに変換,scikit-learnを使うときに必要
#X = iris_df.iloc[:, [0,1,2,3]].values


## データの分割と標準化
トレーニングデータ・テストデータ



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# トレーニングデータとテストデータに分割:トレーニングデータ70%、テストデータ30%
# 乱数を制御するパラメータ random_state は None にすると毎回異なるデータを生成する
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None )

# データの標準化処理 元データを平均0、標準偏差が1のものに変換する正規化法のこと
#インスタンス生成
sc = StandardScaler()

#モデルへのフィット
sc.fit(X_train)

#トレーニングデータとテストデータの標準化処理
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

## モデルへの適用 ・ 結果の算出

In [ ]:
#決定木による学習
from sklearn import tree
from sklearn.metrics import accuracy_score

#モデルのインスタンス生成
tree_model = tree.DecisionTreeClassifier(max_depth=3)

#学習
tree_model.fit(X_train_std,y_train)

#テストデータの適用
predicted = tree_model.predict(X_test_std)

#結果の表示
print('正解率：',accuracy_score(y_test, predicted))

In [ ]:
#トレーニングデータによるフィッティング結果とテストデータによる学習モデルの精度の検証
print('正解率（train):  {:.4f}'.format(tree_model.score(X_train_std, y_train)))
print('正解率（test):  {:.4f}'.format(tree_model.score(X_test_std, y_test)))

## ダミー変数化

In [ ]:
#pandasのデータフレームをarray配列化
from sklearn import preprocessing
X = iris_df[['sepal.length', 'sepal.width', 'petal.length', 'petal.width']]

print(X.head())
print()
# DATA =X.as_matrix() # ← 旧表記方法で、Python3.7以降は 警告またはエラー表示
DATA = X.values
#DATA = X.values.tolist()

print(DATA)

In [ ]:
#ダミー変数化
y_dummy = pd.get_dummies(y)  #, drop_first=True)

print(y_dummy.head())
# ダミー変数の結合
y2 = pd.merge(y, y_dummy, left_index=True, right_index=True)
#y2 = y.join(y_dummy)

print(y2)

## 順序尺度の数値化

In [ ]:
#順序尺度の数値化
size_mapping = {'Virginica': 2, 'Versicolor': 1, 'Setosa': 0}
y3 = y['species'].map(size_mapping)

print(y3.head())
print(y3.tail())

In [ ]:
#名義尺度の数値化 LabelEncoderは、ラベルを0～クラスの種類数n-1の値に変換してくれる、今回は3つのラベルを0,1,2
from sklearn.preprocessing import LabelEncoder
#LabelEncoderのインスタンスを生成
le = LabelEncoder()
#ラベルを覚えさせる
le = le.fit(y)

#ラベルを整数に変換
y_encoded = le.transform(y)

print(y_encoded)

## 回帰による学習と評価

In [ ]:
#回帰による学習とダミー変数,評価は平均二乗誤差 (MSE)と決定係数
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

#トレーニングデータとテストデータに分割
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y3, test_size=0.2, random_state=None )

#インスタンス生成
sc = StandardScaler()

#モデルへのフィット
sc.fit(X_train2)

X_train_std2 = sc.transform(X_train2)
X_test_std2 = sc.transform(X_test2)

#モデルのインスタンス生成と学習(標準化あり)
lr = linear_model.LinearRegression()
lr.fit(X_train_std2, y_train2)

predicted2 = lr.predict(X_test_std2)

print('平均二乗誤差 (MSE)',mean_squared_error(y_test2, predicted2))
print('決定係数 (R2)',r2_score(y_test2, predicted2))

#モデルのインスタンス生成と学習(標準化なし)
lr = linear_model.LinearRegression()
lr.fit(X_train2,y_train2)

predicted2 = lr.predict(X_test2)

print('平均二乗誤差 (MSE)',mean_squared_error(y_test2, predicted2))
print('決定係数 (R2)',r2_score(y_test2, predicted2))


In [ ]:
#小数のまるめ(偶数への丸めであることに注意)
b = np.round(X_train_std2, 2)

#小数の丸める前
print(X_train_std2[0])
#小数の丸め後
print(b[0])

# タイタニックデータを用いたデータ処理

ここでは主に欠損値と説明変数の選択について学習する  
+ 欠損値の確認
+ 欠損値の補間（穴埋め）
+ 説明変数を選択

In [ ]:
#タイタニックデータの読み込み
import pandas as pd

titanic_df = pd.read_csv('/content/drive/MyDrive/DataScience/titanic_train.csv',sep=',')

## 欠損値の確認

In [ ]:
#データの情報表示
print(titanic_df.info())
print()
#各項目の欠損値数の表示
print(titanic_df.isnull().sum())

In [ ]:
titanic_df.head()

## 欠損値の補間（穴埋め）
### 欠損値の基本的な処理方法 : dropna()とfillna()を使う  
+ 欠損値NaN（Not a Number，非数）がひとつでも含まれている行を削除する  
`titanic_df.dropna()`

+ ある行の指定された項目のすべての値が欠損していたら、その行を削除する  
`titanic_df.dropna(subset=['Age','Cabin'], how='all')`

+ すべての欠損値を0で置換する  
`titanic_df.fillna(0)`

+ すべての欠損値を平均で置換する、※数値型のみ  
`titanic_df.fillna(data2.mean())`

+ method = ‘ffill’を使って欠損値を直前の値で補完する数値型にのみ  
`titanic_df.fillna(method = 'ffill')`

+ １次の多項式で補完する。非数値型、前後に値のないデータには動作しない。  
`titanic_df.interpolate(method='polynomial',order=1)`

In [ ]:
# 欠損値がひとつでも含まれている行を削除する
titanic_df_drop1 = titanic_df.dropna()

#データの情報表示
print(titanic_df_drop1.info())
print()                      # 空行の出力で改行

#各項目の欠損値数の表示
print(titanic_df_drop1.isnull().sum())

## 説明変数を選択

In [ ]:
# ある行の指定された項目のすべての値が欠損していたら、その行を削除する all.

#引数how='any'を指定すると，欠損値が一つでも含まれる行が削除される
#デフォルトがhow='any'なので、何も指定しないとこの動作になる
titanic_df_drop2 = titanic_df.dropna(subset=['Age','Cabin'], how='any')

# titanic_df_drop2 = titanic_df.dropna(subset=['Age','Cabin'])
# ↑ デフォルトがhow='any'の為、上のコードと同等

#データの情報表示
print(titanic_df_drop2.info())
print()

#各項目の欠損値数の表示
print(titanic_df_drop2.isnull().sum())
print()
#titanic_df_drop2

In [ ]:
#すべての欠損値を0で置換する
titanic_df_fill1 = titanic_df.fillna(0)

#データの情報表示
print(titanic_df_fill1.info())
print()

#各項目の欠損値数の表示
print(titanic_df_fill1.isnull().sum())
print()
titanic_df_fill1.head()

In [ ]:
#すべての欠損値を平均で置換する、※数値型のみ
titanic_df_fill2 = titanic_df.fillna(titanic_df.mean())

#データの情報表示
print(titanic_df_fill2.info())
print()
#各項目の欠損値数の表示
print(titanic_df_fill2.isnull().sum())
print()
titanic_df_fill2.tail()

In [ ]:
#説明変数と目的変数の選択

#取り出し方 loc:ラベル（変数名）指定 ,iloc:番号指定
Titanic_X = titanic_df.iloc[:, [2,5,9]]
#X = titanic_df.loc[:, ['Age','Fare','Pclass']]

#目的変数の設定
Titanic_y = titanic_df.iloc[:,[1]]
#.loc[:, ['Survived']]

Titanic_X.info()

In [ ]:
#すべての欠損値を平均で置換する、※数値型のみ
Titanic_X_fill = Titanic_X.fillna(Titanic_X.mean())
Titanic_X_fill.info()

In [ ]:
#トレーニングデータとテストデータに分割
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(Titanic_X_fill, Titanic_y, test_size=0.2, random_state=None )

In [ ]:
#決定木による学習
from sklearn import tree
from sklearn.metrics import accuracy_score

#モデルのインスタンス生成
tree_model = tree.DecisionTreeClassifier(max_depth=3)

#学習
tree_model.fit(X_train,y_train)

#トレーニングデータによるフィッティング結果とテストデータによる学習モデルの精度の検証
print('正解率（train):  {:.4f}'.format(tree_model.score(X_train, y_train)))
print('正解率（test):  {:.4f}'.format(tree_model.score(X_test, y_test)))

## sklearnによるデータ処理例,標準化処理
+ 平均値を０，標準偏差を１とする

In [ ]:
#sklearnによるデータ処理例,標準化処理
from sklearn import preprocessing
import numpy as np
X_train = np.array([[10.5, 0.3],[ 0.2, 0.4],[100, 40.4],[0.1, 0.1],[1000.5, 20.2]])
X_scaled = preprocessing.scale(X_train)

print(X_scaled)

# breast-cancerのデータセット

欠損値に文字が挿入されているときの処理方法についての学習  
文字の入れ替えについて学習する。  
Class（病気の重症度）を目的変数として推定を行う。

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

#csvファイルの読み込み
bc_df = pd.read_csv('/content/drive/MyDrive/DataScience/breast-cancer-wisconsin.csv',sep=',')
bc_df.head()

In [ ]:
bc_df.info()

In [ ]:
bc_df.isnull().sum()

In [ ]:
#列ラベル（項目）に属する要素を抽出する．「？」が存在するのが確認できる
#データフレームの中の指定した列の値をユニーク，重複のないデータにする
print(bc_df["Bare Nuclei"].unique())

In [ ]:
#ラベル（項目）に属する要素の数を調べる．？が16個存在しているのがわかる
print(bc_df["Bare Nuclei"].value_counts())

In [ ]:
#？を別の文字に入れ替え：replace
bc_df2 = bc_df.replace("?", "5")
#情報を確認：ただし、Bare Nucleiのデータ型はobjectのまま
bc_df2.info()

In [ ]:
#型の変換(列の指定)
bc_df2 = bc_df2.astype({'Bare Nuclei':'int64'})
#型の変換（全ての列をまとめて変換する場合）
#bc_df2 = bc_df2.astype('int64')
#データの確認
bc_df2.info()

In [ ]:
bc_df2.head()

In [ ]:
#説明変数と目的変数の選択

#取り出し方 loc:ラベル（変数名）指定 ,iloc:番号指定
BC_X = bc_df2.iloc[:, [1,2,3,4,5,6,7,8,9]]
#X = data2.loc[:, ['Age','Fare','Pclass']]

#目的変数の設定
BC_Y = bc_df2.iloc[:,[10]]
#.loc[:, ['Survived']]

BC_X.info()